# Performance dataframes

In [ ]:
import os
import re
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from natsort import index_natsorted

## General parameters

In [ ]:
instance_key = 'instance'

avg_constr_obj_key = 'constr obj'
best_obj_key = 'best obj'
worst_obj_key = 'worst obj'
avg_obj_key = 'obj'

max_time_key = 'max time'
min_time_key = 'min time'
avg_time_key = 'time'

max_iter_key = 'max iter'
min_iter_key = 'min iter'
avg_best_sol_iter_key = 'best found iter'
avg_iter_key = 'iter'

dr_improv_key = 'dr improv (#)'
ls_improv_key = 'ls improv (#)'
best_ls_improv_percent_key = 'best ls improv (%)'
set_part_key = 'sp improv (#)'

best_sol_found_by_key = 'best sol found by'
dr_found_best_sol_key = 'drfb'
ls_found_best_sol_key = 'lsfb'
sp_found_best_sol_key = 'spfb'
cr_found_best_sol_key = 'crfb'

gap_key = 'gap'
preprocess_key = 'preproc'
model_key = 'model time'
variables_key = 'variables'

one_exchange_key = 'one exchange'
one_relocate_key = 'one relocate'
two_exchange_key = 'two exchange'
two_relocate_key = 'two relocate'
post_sched_key = 'postpone scheduled'
sched_post_key = 'schedule postponed'
voyage_exchange_key = 'voyage exchange'

project_path = os.path.dirname(os.path.abspath('.'))
directory_path_alns = '/output/solstorm/alns/performance/'
directory_path_exact = '/output/solstorm/arcflow/performance/'

generate_df = False
run_number = 'fifth'

## Functions

In [ ]:
def map_instance_to_data_alns(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        is_history = split_name[2] == 'history'
        if is_history:
            with open(run_path + file_name) as file:
                history_json = json.load(file)
            
            avg_constr_obj = history_json['construction_heuristic_objective']
            obj = history_json['best_objective']
            time = history_json['runtime']
            it = history_json['number_of_iterations']
            best_it = history_json['best_sol_found_in_iteration']
            set_part_improv = history_json['number_of_improvements_by_set_partitioning']
            ls_improv = history_json['number_of_improvements_by_local_search']
            ls_improv_percent = history_json['best_improvement_local_search']
            dr_improv = history_json['number_of_improvements_by_destroy_repair']
            best_sol_found_by = history_json['best_solution_found_by']
            dr = 1 if best_sol_found_by == 'destroy_repair' else 0
            ls = 1 if best_sol_found_by == 'local_search' else 0
            sp = 1 if best_sol_found_by == 'set_partitioning' else 0
            cr = 1 if best_sol_found_by == 'construction_heuristic' else 0

            if instance_name in instance_to_data:
                data = instance_to_data[instance_name]

                if obj < data[0]:
                    data[0] = obj
                if obj > data[1]:
                    data[1] = obj
                if time > data[4]:
                    data[4] = time
                if time < data[5]:
                    data[5] = time
                if it > data[7]:
                    data[7] = it
                if it < data[8]:
                    data[8] = it
                
                data[2] += obj
                data[3] += avg_constr_obj
                data[6] += time
                data[9] += it
                data[10] += best_it
                data[11] += dr_improv
                data[12] += ls_improv
                data[13] += ls_improv_percent
                data[14] += set_part_improv
                data[15] += dr
                data[16] += ls
                data[17] += sp
                data[18] += cr
                data[19] += 1
            else:
                instance_to_data[instance_name] = [obj, obj, obj, avg_constr_obj,
                                                   time, time, time, 
                                                   it, it, it, best_it,
                                                   dr_improv, ls_improv, ls_improv_percent, set_part_improv,
                                                   dr, ls, sp, cr,
                                                   1]
    return instance_to_data

def generate_run_df_alns(run_name):
    run_path = project_path + directory_path_alns + run_name
    instance_to_data = map_instance_to_data_alns(run_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               best_obj_key, worst_obj_key, avg_obj_key, avg_constr_obj_key,
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_iter_key, avg_best_sol_iter_key,
                               dr_improv_key, ls_improv_key, best_ls_improv_percent_key, set_part_key,
                               dr_found_best_sol_key, ls_found_best_sol_key, sp_found_best_sol_key, cr_found_best_sol_key])
    
    for instance in instance_to_data:
        data = instance_to_data[instance]
        
        nbr_sims = data[19]
        if nbr_sims != 5:
            print(f'{instance} DEVIATES IN SIMULATIONS!')
        
        best_objective = data[0]
        worst_objective = data[1]
        avg_objective = data[2] / nbr_sims
        avg_constr_objective = data[3] / nbr_sims
        max_time = data[4]
        min_time = data[5]
        avg_time = data[6] / nbr_sims
        max_iter = data[7]
        min_iter = data[8]
        avg_iter = data[9] / nbr_sims
        avg_best_sol_iter = data[10] / nbr_sims
        avg_dr_improv = data[11] / nbr_sims
        avg_ls_improv = data[12] / nbr_sims
        avg_ls_improv_percent = data[13] / nbr_sims
        avg_set_part_improv = data[14] / nbr_sims
        dr_found_best_sol = data[15]
        ls_found_best_sol = data[16]
        sp_found_best_sol = data[17]
        cr_found_best_sol = data[18]
        
        row = pd.Series({instance_key: instance, 
                         best_obj_key: best_objective,
                         worst_obj_key: worst_objective,
                         avg_obj_key: avg_objective,
                         avg_constr_obj_key: avg_constr_objective,
                         max_time_key: max_time,
                         min_time_key: min_time,
                         avg_time_key: avg_time, 
                         max_iter_key: max_iter,
                         min_iter_key: min_iter,
                         avg_iter_key: avg_iter,
                         avg_best_sol_iter_key: avg_best_sol_iter,
                         dr_improv_key: avg_dr_improv,
                         ls_improv_key: avg_ls_improv,
                         best_ls_improv_percent_key: avg_ls_improv_percent,
                         set_part_key: avg_set_part_improv,
                         dr_found_best_sol_key: dr_found_best_sol,
                         ls_found_best_sol_key: ls_found_best_sol,
                         sp_found_best_sol_key: sp_found_best_sol,
                         cr_found_best_sol_key: cr_found_best_sol})
        
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          avg_constr_obj_key: df[avg_constr_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          avg_best_sol_iter_key: df[avg_best_sol_iter_key].mean(),
                          dr_improv_key: df[dr_improv_key].mean(),
                          ls_improv_key: df[ls_improv_key].mean(),
                          best_ls_improv_percent_key: df[best_ls_improv_percent_key].mean(),
                          set_part_key: df[set_part_key].mean(),
                          dr_found_best_sol_key: df[dr_found_best_sol_key].mean(),
                          ls_found_best_sol_key: df[ls_found_best_sol_key].mean(),
                          sp_found_best_sol_key: df[sp_found_best_sol_key].mean(),
                          cr_found_best_sol_key: df[cr_found_best_sol_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def aggregate_df_by_instance_group_alns(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        best_obj = row[best_obj_key]
        worst_obj = row[worst_obj_key]
        avg_obj = row[avg_obj_key]
        avg_constr_obj = row[avg_constr_obj_key]
        max_time = row[max_time_key]
        min_time = row[min_time_key]
        avg_time = row[avg_time_key]
        max_iter = row[max_iter_key]
        min_iter = row[min_iter_key]
        avg_iter = row[avg_iter_key]
        avg_best_sol_iter = row[avg_best_sol_iter_key]
        avg_dr_improv = row[dr_improv_key]
        avg_ls_improv = row[ls_improv_key]
        avg_ls_improv_percent = row[best_ls_improv_percent_key]
        avg_set_part_improv = row[set_part_key]
        avg_dr_found_best_sol = row[dr_found_best_sol_key]
        avg_ls_found_best_sol = row[ls_found_best_sol_key]
        avg_sp_found_best_sol = row[sp_found_best_sol_key]
        avg_cr_found_best_sol = row[cr_found_best_sol_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += best_obj
            data[1] += worst_obj
            data[2] += avg_obj
            data[3] += avg_constr_obj
            data[4] += max_time
            data[5] += min_time
            data[6] += avg_time
            data[7] += max_iter
            data[8] += min_iter
            data[9] += avg_iter
            data[10] += avg_best_sol_iter
            data[11] += avg_dr_improv
            data[12] += avg_ls_improv
            data[13] += avg_ls_improv_percent
            data[14] += avg_set_part_improv
            data[15] += avg_dr_found_best_sol
            data[16] += avg_ls_found_best_sol
            data[17] += avg_sp_found_best_sol
            data[18] += avg_cr_found_best_sol
            data[19] += 1  # Number of times encountered instance size
        else:
            data = [best_obj, worst_obj, avg_obj, avg_constr_obj,
                    max_time, min_time, avg_time, 
                    max_iter, min_iter, avg_iter, avg_best_sol_iter,
                    avg_dr_improv, avg_ls_improv, avg_ls_improv_percent, avg_set_part_improv,
                    avg_dr_found_best_sol, avg_ls_found_best_sol, avg_sp_found_best_sol, avg_cr_found_best_sol,
                    1]
            instance_size_to_data[instance_size] = data

    df = pd.DataFrame(columns=['instance_group', 
                               best_obj_key, worst_obj_key, avg_obj_key, avg_constr_obj_key,
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_iter_key, avg_best_sol_iter_key,
                               dr_improv_key, ls_improv_key, best_ls_improv_percent_key, set_part_key,
                               dr_found_best_sol_key, ls_found_best_sol_key, sp_found_best_sol_key, cr_found_best_sol_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[19]
        if nbr_sims != 5:
            print(f'{instance_size} INSTANCE_SIZE DEVIATES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         best_obj_key: data[0] / nbr_sims,
                         worst_obj_key: data[1] / nbr_sims,
                         avg_obj_key: data[2] / nbr_sims,
                         avg_constr_obj_key: data[3] / nbr_sims,
                         max_time_key: data[4] / nbr_sims,
                         min_time_key: data[5] / nbr_sims,
                         avg_time_key: data[6] / nbr_sims, 
                         max_iter_key: data[7] / nbr_sims,
                         min_iter_key: data[8] / nbr_sims,
                         avg_iter_key: data[9] / nbr_sims,
                         avg_best_sol_iter_key: data[10] / nbr_sims,
                         dr_improv_key: data[11] / nbr_sims,
                         ls_improv_key: data[12] / nbr_sims,
                         best_ls_improv_percent_key: data[13] / nbr_sims,
                         set_part_key: data[14] / nbr_sims,
                         dr_found_best_sol_key: data[15] / nbr_sims,
                         ls_found_best_sol_key: data[16] / nbr_sims,
                         sp_found_best_sol_key: data[17] / nbr_sims,
                         cr_found_best_sol_key: data[18] / nbr_sims,})
    
        df = df.append(row, ignore_index=True)
        
    mean_row = pd.Series({'instance_group': 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          avg_constr_obj_key: df[avg_constr_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          avg_best_sol_iter_key: df[avg_best_sol_iter_key].mean(),
                          dr_improv_key: df[dr_improv_key].mean(),
                          ls_improv_key: df[ls_improv_key].mean(),
                          best_ls_improv_percent_key: df[best_ls_improv_percent_key].mean(),
                          set_part_key: df[set_part_key].mean(),
                          dr_found_best_sol_key: df[dr_found_best_sol_key].mean(),
                          ls_found_best_sol_key: df[ls_found_best_sol_key].mean(),
                          sp_found_best_sol_key: df[sp_found_best_sol_key].mean(),
                          cr_found_best_sol_key: df[cr_found_best_sol_key].mean(),})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def map_instance_to_data_exact(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        with open(run_path + file_name) as file:
            exact_json = json.load(file)

        obj = exact_json['objective']['objective_bound']
        gap = exact_json['objective']['optimality_gap']
        preprocess_runtime = exact_json['runtime']['preprocess_runtime']
        model_runtime = exact_json['runtime']['model_runtime']
        variables = exact_json['variables']['number_of_variables']
        
        if instance_name in instance_to_data:
            print('Multiple versions of same instance!')
        
        instance_to_data[instance_name] = [obj, gap, preprocess_runtime, model_runtime, variables]
    
    return instance_to_data

def generate_run_df_exact(run_name):
    run_path = project_path + directory_path_exact + run_name
    instance_to_data = map_instance_to_data_exact(run_path)

    df = pd.DataFrame(columns=[instance_key, avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    for instance in instance_to_data:
        data = instance_to_data[instance]
        obj = data[0]
        gap = data[1]
        preprocess_runtime = data[2]
        model_runtime = data[3]
        variables = data[4]
        row = pd.Series({instance_key: instance,
                         avg_obj_key: obj,
                         gap_key: gap,
                         preprocess_key: preprocess_runtime,
                         model_key: model_runtime,
                         variables_key: variables})
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(),
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def merge_dfs(dfs, drop):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, avg_constr_obj_key,
                 max_time_key, min_time_key, 
                 max_iter_key, min_iter_key], 
                axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    if drop:
        li = [i for i in range(5, len(df_total.columns), 5)]
        df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

def aggregate_df_by_instance_group_exact(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        obj = row[avg_obj_key]
        gap = row[gap_key]
        preprocess_runtime = row[preprocess_key]
        model_runtime = row[model_key]
        variables = row[variables_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += obj
            data[1] += gap
            data[2] += preprocess_runtime
            data[3] += model_runtime
            data[4] += variables
            data[5] += 1 # Number of times encountered instance size
        else:
            instance_size_to_data[instance_size] = [obj, gap, preprocess_runtime, model_runtime, variables, 1]

    df = pd.DataFrame(columns=['instance_group', avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[5]
        if nbr_sims < 5:
            print(f'{instance_size} INSTANCE_SIZE LESS THAN FIVE INSTANCES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         avg_obj_key: data[0] / nbr_sims,
                         gap_key: data[1] / nbr_sims,
                         preprocess_key: data[2] / nbr_sims, 
                         model_key: data[3] / nbr_sims,
                         variables_key: data[4] / nbr_sims})
    
        df = df.append(row, ignore_index=True)

        
    mean_row = pd.Series({'instance_group': 'Mean values', 
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(), 
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def map_instance_to_data_lso(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        is_history = split_name[2] == 'history'
        if is_history:
            with open(run_path + file_name) as file:
                history_json = json.load(file)
            
            nbr_improv_one_exchange = history_json['number_of_improvements_by_local_search_operators']['one_exchange']
            nbr_improv_one_relocate = history_json['number_of_improvements_by_local_search_operators']['one_relocate']
            nbr_improv_two_exchange = history_json['number_of_improvements_by_local_search_operators']['two_exchange']
            nbr_improv_two_relocate = history_json['number_of_improvements_by_local_search_operators']['two_relocate']
            nbr_improv_post_sched = history_json['number_of_improvements_by_local_search_operators']['postpone_scheduled']
            nbr_improv_sched_post = history_json['number_of_improvements_by_local_search_operators']['schedule_postponed']
            nbr_improv_voy_exchange = history_json['number_of_improvements_by_local_search_operators']['voyage_exchange']
            
            if instance_name in instance_to_data:
                data = instance_to_data[instance_name]
                data[0] += nbr_improv_one_exchange
                data[1] += nbr_improv_one_relocate
                data[2] += nbr_improv_two_exchange
                data[3] += nbr_improv_two_relocate
                data[4] += nbr_improv_post_sched
                data[5] += nbr_improv_sched_post
                data[6] += nbr_improv_voy_exchange
                data[7] += 1
            else:
                instance_to_data[instance_name] = [nbr_improv_one_exchange,
                                                   nbr_improv_one_relocate,
                                                   nbr_improv_two_exchange,
                                                   nbr_improv_two_relocate,
                                                   nbr_improv_post_sched,
                                                   nbr_improv_sched_post,
                                                   nbr_improv_voy_exchange,
                                                   1]
    return instance_to_data

def generate_lso_df(run_name):
    run_path = project_path + directory_path_alns + run_name
    instance_to_data = map_instance_to_data_lso(run_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               one_exchange_key, one_relocate_key, 
                               two_exchange_key, two_relocate_key,
                               post_sched_key, sched_post_key,
                               voyage_exchange_key])
    
    for instance in instance_to_data:
        data = instance_to_data[instance]
        
        nbr_sims = data[7]
        
        nbr_improv_one_exchange = data[0] / nbr_sims
        nbr_improv_one_relocate = data[1] / nbr_sims
        nbr_improv_two_exchange = data[2] / nbr_sims
        nbr_improv_two_relocate = data[3] / nbr_sims
        nbr_improv_post_sched = data[4] / nbr_sims
        nbr_improv_sched_post = data[5] / nbr_sims
        nbr_improv_voy_exchange = data[6] / nbr_sims
        
        row = pd.Series({instance_key: instance,
                         one_exchange_key: nbr_improv_one_exchange,
                         one_relocate_key: nbr_improv_one_relocate,
                         two_exchange_key: nbr_improv_two_exchange,
                         two_relocate_key: nbr_improv_two_relocate,
                         post_sched_key: nbr_improv_post_sched,
                         sched_post_key: nbr_improv_sched_post,
                         voyage_exchange_key: nbr_improv_voy_exchange})

        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          one_exchange_key: df[one_exchange_key].mean(),
                          one_relocate_key: df[one_relocate_key].mean(),
                          two_exchange_key: df[two_exchange_key].mean(),
                          two_relocate_key: df[two_relocate_key].mean(),
                          post_sched_key: df[post_sched_key].mean(),
                          sched_post_key: df[sched_post_key].mean(),
                          voyage_exchange_key: df[voyage_exchange_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def load_df(file_name):
    run_df = pd.read_pickle(f'dataframes/performance/{file_name}')
    # run_df = sort_df(run_df, sort_column)
    return run_df

def merge_dfs(dfs, drop):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, avg_constr_obj_key,
                 max_time_key, min_time_key, 
                 max_iter_key, min_iter_key, avg_best_sol_iter_key], 
                axis=1, inplace=True)
        if best_sol_found_by_key in df:
            df.drop([best_sol_found_by_key], axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    if drop:
        li = [i for i in range(12, len(df_total.columns), 12)]
        df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

## ALNS baseline

In [ ]:
if generate_df:
    run_baseline_name = f'{run_number}/baseline/'
    run_baseline_df = generate_run_df_alns(run_baseline_name)
    run_baseline_agg_df = aggregate_df_by_instance_group_alns(run_baseline_df)

    run_baseline_file_name = f'dataframes/performance/baseline.pkl'
    run_baseline_agg_file_name = f'dataframes/performance/baseline_agg.pkl'
    run_baseline_df.to_pickle(run_baseline_file_name)
    run_baseline_agg_df.to_pickle(run_baseline_agg_file_name)

In [ ]:
run_baseline_df = load_df('baseline.pkl')
run_baseline_df

In [ ]:
run_baseline_agg_df = load_df('baseline_agg.pkl')
run_baseline_agg_df

## Sequential ALNS

In [ ]:
if generate_df:
    run_sequential_name = 'fifth/sequential/'
    run_sequential_df = generate_run_df_alns(run_sequential_name)
    run_sequential_agg_df = aggregate_df_by_instance_group_alns(run_sequential_df)

    run_sequential_file_name = f'dataframes/performance/sequential.pkl'
    run_sequential_agg_file_name = f'dataframes/performance/sequential_agg.pkl'
    run_sequential_df.to_pickle(run_sequential_file_name)
    run_sequential_agg_df.to_pickle(run_sequential_agg_file_name)

In [ ]:
run_sequential_df = load_df('sequential.pkl')
run_sequential_df

In [ ]:
run_sequential_agg_df = load_df('sequential_agg.pkl')
run_sequential_agg_df

## LNS

In [ ]:
if generate_df:
    run_lns_name = f'{run_number}/lns/'
    run_lns_df = generate_run_df_alns(run_lns_name)
    run_lns_agg_df = aggregate_df_by_instance_group_alns(run_lns_df)

    run_lns_file_name = f'dataframes/performance/lns.pkl'
    run_lns_agg_file_name = f'dataframes/performance/lns_agg.pkl'
    run_lns_df.to_pickle(run_lns_file_name)
    run_lns_agg_df.to_pickle(run_lns_agg_file_name)

In [ ]:
run_lns_df = load_df('lns.pkl')
run_lns_df

In [ ]:
run_lns_agg_df = load_df('lns_agg.pkl')
run_lns_agg_df

## ALNS + local search

In [ ]:
if generate_df:
    run_ls_name = f'{run_number}/ls/'
    run_ls_df = generate_run_df_alns(run_ls_name)
    run_ls_agg_df = aggregate_df_by_instance_group_alns(run_ls_df)

    run_ls_file_name = f'dataframes/performance/ls.pkl'
    run_ls_agg_file_name = f'dataframes/performance/ls_agg.pkl'
    run_ls_df.to_pickle(run_ls_file_name)
    run_ls_agg_df.to_pickle(run_ls_agg_file_name)

In [ ]:
run_ls_df = load_df('ls.pkl')
run_ls_df

In [ ]:
run_ls_agg_df = load_df('ls_agg.pkl')
run_ls_agg_df

## ALNS + set partitioning

In [ ]:
if generate_df: 
    run_sp_name = f'{run_number}/sp/'
    run_sp_df = generate_run_df_alns(run_sp_name)
    run_sp_agg_df = aggregate_df_by_instance_group_alns(run_sp_df)

    run_sp_file_name = f'dataframes/performance/sp.pkl'
    run_sp_agg_file_name = f'dataframes/performance/sp_agg.pkl'
    run_sp_df.to_pickle(run_sp_file_name)
    run_sp_agg_df.to_pickle(run_sp_agg_file_name)

In [ ]:
run_sp_df = load_df('sp.pkl')
run_sp_df

In [ ]:
run_sp_agg_df = load_df('sp_agg.pkl')
run_sp_agg_df

## ALNS + local search + set partitioning

In [ ]:
if generate_df:
    run_lssp_name = f'{run_number}/lssp/'
    run_lssp_df = generate_run_df_alns(run_lssp_name)
    run_lssp_agg_df = aggregate_df_by_instance_group_alns(run_lssp_df)

    run_lssp_file_name = f'dataframes/performance/lssp.pkl'
    run_lssp_agg_file_name = f'dataframes/performance/lssp_agg.pkl'
    run_lssp_df.to_pickle(run_lssp_file_name)
    run_lssp_agg_df.to_pickle(run_lssp_agg_file_name)

In [ ]:
run_lssp_df = load_df('lssp.pkl')
run_lssp_df

In [ ]:
run_lssp_agg_df = load_df('lssp_agg.pkl')
run_lssp_agg_df

## Exact solver 3600

In [ ]:
if generate_df:
    run_exact_3600_name = '3600/results/'
    run_exact_3600_df = generate_run_df_exact(run_exact_3600_name)
    run_exact_3600_agg_df = aggregate_df_by_instance_group_exact(run_exact_3600_df)

    run_exact_3600_file_name = f'dataframes/performance/exact_3600.pkl'
    run_exact_3600_agg_file_name = f'dataframes/performance/exact_3600_agg.pkl'
    run_exact_3600_df.to_pickle(run_exact_3600_file_name)
    run_exact_3600_agg_df.to_pickle(run_exact_3600_agg_file_name)

In [ ]:
run_exact_3600_df = load_df('exact_3600.pkl')
run_exact_3600_df

In [ ]:
run_exact_3600_agg_df = load_df('exact_3600_agg.pkl')
run_exact_3600_agg_df

## Exact solver 600

In [ ]:
if generate_df:
    run_exact_600_name = '600/results/'
    run_exact_600_df = generate_run_df_exact(run_exact_600_name)
    run_exact_600_agg_df = aggregate_df_by_instance_group_exact(run_exact_600_df)

    run_exact_600_file_name = f'dataframes/performance/exact_600.pkl'
    run_exact_600_agg_file_name = f'dataframes/performance/exact_600_agg.pkl'
    run_exact_600_df.to_pickle(run_exact_600_file_name)
    run_exact_600_agg_df.to_pickle(run_exact_600_agg_file_name)

In [ ]:
run_exact_600_df = load_df('exact_600.pkl')
run_exact_600_df

In [ ]:
run_exact_600_agg_df = load_df('exact_600_agg.pkl')
run_exact_600_agg_df

## Parallel vs. sequential heuristics

In [ ]:
baseline_sequential_df = merge_dfs([run_baseline_df, run_sequential_df], True)
baseline_sequential_agg_df = merge_dfs([run_baseline_agg_df, run_sequential_agg_df], True)

In [ ]:
baseline_sequential_df

In [ ]:
baseline_sequential_agg_df

## ALNS vs. LNS

In [ ]:
baseline_lns_df = merge_dfs([run_baseline_df, run_lns_df], True)
baseline_lns_agg_df = merge_dfs([run_baseline_agg_df, run_lns_agg_df], True)

In [ ]:
baseline_lns_df

In [ ]:
baseline_lns_agg_df

## ALNS vs. ALNS + local search

In [ ]:
baseline_ls_df = merge_dfs([run_baseline_df, run_ls_df], True)
baseline_ls_agg_df = merge_dfs([run_baseline_agg_df, run_ls_agg_df], True)

In [ ]:
baseline_ls_df

In [ ]:
baseline_ls_agg_df

## ALNS vs. ALNS + set partitioning

In [ ]:
baseline_sp_df = merge_dfs([run_baseline_df, run_sp_df], True)
baseline_sp_agg_df = merge_dfs([run_baseline_agg_df, run_sp_agg_df], True)

In [ ]:
baseline_sp_df

In [ ]:
baseline_sp_agg_df

## ALNS vs. ALNS + local search + set partitioning

In [ ]:
baseline_lssp_df = merge_dfs([run_baseline_df, run_lssp_df], True)
baseline_lssp_agg_df = merge_dfs([run_baseline_agg_df, run_lssp_agg_df], True)

In [ ]:
baseline_lssp_df

In [ ]:
baseline_lssp_agg_df

## Best ALNS vs exact solver

In [ ]:
# alns_exact_df = merge_dfs([run_lssp_df, ])

## Local search operators

In [ ]:
if generate_df:
    run_ls_name = f'{run_number}/ls/'
    lso_df = generate_lso_df(run_ls_name)
    lso_file_name = f'dataframes/performance/lso.pkl'
    lso_df.to_pickle(lso_file_name)

In [ ]:
lso_df = load_df('lso.pkl')
lso_df